In [1]:
# https://www.youtube.com/watch?v=1lwddP0KUEg

In [2]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [3]:
lemmitizer = WordNetLemmatizer()

In [4]:
intents = json.loads(open('layan_file.json').read())

In [5]:
intents

{'intents': [{'tag': 'google',
   'patterns': ['google', 'search', 'internet'],
   'responses': ['Redirecting to Google...'],
   'context': None},
  {'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day',
    'Namaste',
    'yo'],
   'responses': ['Hello',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Get lost',
    'Till next time',
    'bbye'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info

In [6]:
words = []
classes = []
documents = []
ignore_letters = ['.', ',', '?', '!']

#### in order to make a chatbot, we will some data, we used an intent.json file that has data categorized in tags and patterns and repsonses, most people use intent.json files, anyway, we will need to tokenize the data in the file

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
for i in intents['intents']:
    print(i['patterns'])

['google', 'search', 'internet']
['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day', 'Namaste', 'yo']
['Bye', 'See you later', 'Goodbye', 'Get lost', 'Till next time', 'bbye']
['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me']
[]
['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered']
['Tell me a joke', 'Joke', 'Make me laugh']
['Who are you', 'what are you']
['What is the time', 'what is the date', 'date', 'time', 'tell me the date', 'day', 'what day is is today']
['Whats up', 'Wazzup', 'How are you', 'sup', 'How you doing']
['haha', 'lol', 'rofl', 'lmao', 'thats funny']
['Who made you', 'who designed you', 'who programmed you']
['you are dumb', 'shut up', 'idiot']
['what are you doing', 'what are you upto']
['Awesome', 'Great', 'I know', 'ok', 'yeah']
['temperature', 'weather', 'how hot is it']
['who is he', 'who is that', 'who is karan', 'karan ma

#### the words list that we have defined earlier will now have all the patterns in the intent.json file
#### the classes list will have the tags
#### and the document list will have each word with their co-responding tag

In [9]:
# we lemmitize then we oreder and remove duplicates by placing 'words' in a set
words = [lemmitizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

# we do the same thing for classes just to be sure
classes = sorted(set(classes))

In [10]:
# and now we save the results in a pickle file
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

#### and now the fun part... Machine Learning Baby!!

#### i don't fully understand what's going on here, but I think we are getting the 'words' list from the 'documents' tuple, and giving each word a number, if it exist it's 1, if it doesn't it's 0

In [11]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    
    # getting the words inside of the document tuple and making them lower case
    word_patterns = doc[0]
    word_patterns = [lemmitizer.lemmatize(word.lower()) for word in word_patterns]

    # from the wrods list that we defined above, if it exist in the word_patterns we give the bag a value of 1 if it doesn't we give it a value of 0
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

#### neural network stuff

In [12]:
random.shuffle(training)
training = np.array(training, dtype='object')

train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [13]:
# random thought, get 10 intent.json files and slap them together into one file

In [14]:
# now we build the model

model = Sequential()
# model.add(Dense(128, input_shape=len(train_x[0]), activation='relu') #128 is the number of neurons
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nseterov=True)
sgd = SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

model.save('chatbot_model.model', hist)
print('done!')


Epoch 1/200


44/44 [==============================] - 1s 2ms/step - loss: 4.7861 - accuracy: 0.0230
Epoch 2/200
44/44 [==============================] - 0s 2ms/step - loss: 4.7312 - accuracy: 0.0323
Epoch 3/200
44/44 [==============================] - 0s 2ms/step - loss: 4.6905 - accuracy: 0.0461
Epoch 4/200
44/44 [==============================] - 0s 2ms/step - loss: 4.6314 - accuracy: 0.0461
Epoch 5/200
44/44 [==============================] - 0s 2ms/step - loss: 4.5894 - accuracy: 0.0323
Epoch 6/200
44/44 [==============================] - 0s 2ms/step - loss: 4.4762 - accuracy: 0.0323
Epoch 7/200
44/44 [==============================] - 0s 2ms/step - loss: 4.3968 - accuracy: 0.0737
Epoch 8/200
44/44 [==============================] - 0s 2ms/step - loss: 4.3260 - accuracy: 0.0645
Epoch 9/200
44/44 [==============================] - 0s 3ms/step - loss: 4.2203 - accuracy: 0.0876
Epoch 10/200
44/44 [==============================] - 0s 2ms/step - loss: 4.1473 - accuracy: 0.0876
Epoch 

INFO:tensorflow:Assets written to: chatbot_model.model\assets


done!


#### and now finally the chatbot part of the code

In [15]:
from tensorflow.keras.models import load_model

In [16]:
lemmatizer = WordNetLemmatizer()
intent = json.loads(open('intents.json').read())

In [17]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

In [18]:
model = load_model('chatbot_model.model')

In [19]:
def clean_up_sentance(sentance):
    sentance_words = nltk.word_tokenize(sentance)
    sentance_words = [lemmatizer.lemmatize(word) for word in sentance_words]
    return sentance_words

def bag_of_words(sentance):
    sentance = clean_up_sentance(sentance)
    bag = [0] * len(words)
    for w in sentance:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentance):
    bow = bag_of_words(sentance)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    result = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    result.sort(key=lambda x: x[1], reverse=True)
    retrun_list = []
    for r in result:
        retrun_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return retrun_list

def get_response(intent_list, intent_json):
    tag = intent_list[0]['intent']
    list_of_intents = intent_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [22]:
while True:
    message = input('')
    if message=='bye':
        print('bye bye')
        break
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

 mk


1/1 [==============================] - 0s 19ms/step
Have a nice day


 ml


1/1 [==============================] - 0s 19ms/step
Bye! Come back again soon.


 machine learning


...

 your welcome


1/1 [==============================] - 0s 20ms/step
Have a nice day


 data


1/1 [==============================] - 0s 19ms/step


IndexError: list index out of range